**데이터 로드**

In [1]:
%pip install tensorflow==2.9.0
%pip install tensorflow-text==2.9.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pandas
%pip install scikit-learn==1.2.2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
      --------------------------------------- 0.2/8.3 MB 6.1 MB/s eta 0:00:02
     -- ------------------------------------- 0.6/8.3 MB 7.2 MB/s eta 0:00:02
     --- ------------------------------------ 0.7/8.3 MB 5.2 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/8.3 MB 6.4 MB/s eta 0:00:02
     ------- -------------------------------- 1.6/8.3 MB 7.1 MB/s eta 0:00:01
     --------- ------------------------------ 1.9/8.3 MB 7.2 MB/s eta 0:00:01
     ---------- ----------------------------- 2.3/8.3 MB 7.2 MB/s eta 0:00:01
     ------------ --------------------------- 2.7/8.3 MB 7.3 MB/s eta 0:00:01
     -------------- ------------------------- 3.1/8.3 MB 7.5 MB/s eta 0:00:01
     ----------------- ---------------------- 3.5/8.3 MB 7.8 MB/s eta 0:00:01
     ------------------ --------------------- 3.9/8.3 MB 7.8 MB/s eta 0:00:01
     -------------------- ------------------- 4.2/8.3 MB 7.7 MB/s eta 0


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from concurrent.futures import ThreadPoolExecutor
from itertools import product
import multiprocessing

In [ ]:
# 시작 방법을 'forkserver'로 설정
multiprocessing.set_start_method('forkserver', force=True)

In [ ]:
# 데이터 로드
mbti = pd.read_csv('resource/MBTI 500.csv')
movies = pd.read_csv('resource/movie_meta_data_final.csv', encoding='utf-8')

In [4]:
from tensorflow import keras

In [ ]:

model_path = 'resource/labse_model.h5'
labse_model = keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

**LaBSE 모델 로드 및 임베딩 생성**

In [ ]:
# LaBSE 모델 로드
encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1")
preprocessor = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1")

# 텍스트를 고차원 벡터로 인코딩하는 모델 구성
sentences = tf.keras.layers.Input(shape=(), dtype=tf.string, name="sentences")
encoder_inputs = preprocessor(sentences)
sentence_representation = encoder(encoder_inputs)["pooled_output"]
normalized_sentence_representation = tf.nn.l2_normalize(sentence_representation, axis=-1)
labse_model = tf.keras.Model(sentences, normalized_sentence_representation)
labse_model.compile()

# 임베딩을 생성하는 함수
def embed_sentence(sentence):
    return labse_model(tf.constant([sentence]))[0].numpy().astype(np.float64)

In [ ]:
# 임베딩을 생성하는 함수
def embed_sentence(sentence):
    return labse_model(tf.constant([sentence]))[0].numpy().astype(np.float64)

In [ ]:
mbti_embeddings_dict

**영화 데이터 전처리 및 임베딩 생성**

In [ ]:
# 병렬 처리를 이용하여 임베딩 생성
def parallel_embedding(texts):
    with ThreadPoolExecutor() as executor:
        embeddings = list(executor.map(embed_sentence, texts))
    return np.array(embeddings).astype(np.float64)

# MBTI 임베딩 병렬 처리
mbti_posts = mbti.groupby('type')['posts'].apply(lambda posts: ' '.join(posts)).tolist()
mbti_embeddings = parallel_embedding(mbti_posts)
mbti_index = mbti.groupby('type').groups.keys()
mbti_embeddings_dict = dict(zip(mbti_index, mbti_embeddings))

# 한국어 영화 줄거리 임베딩 병렬 처리
movie_texts = movies['overview'].tolist()
movie_embeddings = parallel_embedding(movie_texts)
movie_index = movies['name'].tolist()
movie_embeddings_dict = dict(zip(movie_index, movie_embeddings))

# 영화 장르 원핫 인코딩
genres = movies['genres'].str.get_dummies(sep=',')
cbf_model_input = np.hstack([genres.values, movies['star_avg'].values.reshape(-1, 1)])
cbf_scaler = StandardScaler()
cbf_model_input_scaled = cbf_scaler.fit_transform(cbf_model_input)

In [ ]:
import pickle

with open('mbti_embeddings_dict.pkl', 'wb') as f:
    pickle.dump(mbti_embeddings_dict, f)

In [ ]:
import pickle

with open('movie_embeddings_dict.pkl', 'wb') as f:
    pickle.dump(movie_embeddings_dict, f)

**CBF 모델 하이퍼파라미터 최적화**

In [ ]:
# 하이퍼파라미터 그리드 설정 -> 필요 없음 
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.8, 1.0]
}

# 그리드 서치 설정
gbm = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

# 학습 데이터와 평가 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(cbf_model_input_scaled, movies['star_avg'].values, test_size=0.2, random_state=42)

# 그리드 서치 수행
grid_search.fit(X_train, y_train)

# 최적의 모델 출력
best_gbm = grid_search.best_estimator_
print(f'Best GBM Model: {best_gbm}')

# 파라미터 

In [ ]:
import sklearn.external.joblib as extjoblib

In [5]:
import joblib

In [ ]:
import pickle

In [ ]:
file_path = 'resource/mbti_embeddings_dict.pkl'

In [ ]:
# 데이터 로드
with open(file_path, 'rb') as f:
	data = pickle.load(f)

In [6]:
# 최적의 모델 로드
best_gbm = joblib.load('resource/best_gbm_model.pkl')

**MBTI 기반 추천, 선호 영화 유사도 기반 추천, 결합 추천 시스템 정의**

In [ ]:
# 영화와 MBTI 유형 간의 유사도 계산 함수
def calculate_similarity(movie_embedding, user_embedding):
    from numpy import dot
    from numpy.linalg import norm
    return dot(movie_embedding, user_embedding) / (norm(movie_embedding) * norm(user_embedding))

# 1. MBTI 임베딩 기반 추천
def recommend_movies_by_mbti(user_embedding, movie_embeddings_dict, top_n=100):
    recommended_movies = []
    for movie, embedding in movie_embeddings_dict.items():
        similarity = calculate_similarity(embedding, user_embedding)
        recommended_movies.append((movie, similarity))
    recommended_movies = sorted(recommended_movies, key=lambda x: x[1], reverse=True)
    return recommended_movies[:top_n]

# 2. 선호 영화 유사도 기반 추천
def recommend_similar_movies(preferred_movies, movie_embeddings_dict, top_n=100):
    similar_movies = {}
    for movie in preferred_movies:
        movie_embedding = movie_embeddings_dict[movie]
        for other_movie, embedding in movie_embeddings_dict.items():
            if other_movie not in preferred_movies:
                similarity = calculate_similarity(embedding, movie_embedding)
                if other_movie not in similar_movies:
                    similar_movies[other_movie] = 0
                similar_movies[other_movie] += similarity
    similar_movies = sorted(similar_movies.items(), key=lambda x: x[1], reverse=True)
    return similar_movies[:top_n]

# MSE 계산 함수
def calculate_mse(recommendations, movies, model, cbf_model_input_scaled):
    movie_indices = [movies[movies['name'] == movie].index[0] for movie in recommendations]
    movie_features = cbf_model_input_scaled[movie_indices]
    true_ratings = movies.loc[movie_indices, 'star_avg'].values
    predicted_ratings = model.predict(movie_features)
    mse = mean_squared_error(true_ratings, predicted_ratings)
    return mse

# 결합 추천 시스템
def recommend_movies_combined(user_embedding, preferred_movies, movie_embeddings_dict, model,
                              weight_mbti=0.3, weight_similar=0.3, weight_model=0.4, top_n=20):
    mbti_recommendations = recommend_movies_by_mbti(user_embedding, movie_embeddings_dict, top_n=100)
    similar_movie_recommendations = recommend_similar_movies(preferred_movies, movie_embeddings_dict, top_n=100)

    combined_recommendations = {}

    for movie, score in mbti_recommendations:
        if movie not in combined_recommendations:
            combined_recommendations[movie] = 0
        combined_recommendations[movie] += weight_mbti * score

    for movie, score in similar_movie_recommendations:
        if movie not in combined_recommendations:
            combined_recommendations[movie] = 0
        combined_recommendations[movie] += weight_similar * score

    movie_indices = [movies[movies['name'] == movie].index[0] for movie in combined_recommendations.keys()]
    if movie_indices:
        movie_features = cbf_model_input_scaled[movie_indices]
        model_scores = model.predict(movie_features).flatten()

        for movie, model_score in zip(combined_recommendations.keys(), model_scores):
            combined_recommendations[movie] += weight_model * model_score

    filtered_recommendations = {movie: score for movie, score in combined_recommendations.items()
                                if movies.loc[movies['name'] == movie, 'star_avg'].values[0] >= 3.3}

    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie, _ in final_recommendations[:top_n]]

**최적의 가중치 찾기**

In [ ]:
# 예시 사용자 입력 데이터
new_user_mbti = "INTP"
new_user_preferred_movies = ["해리포터와 불의 잔", "스파이더맨: 노 웨이 홈"]

# 예시 사용자 임베딩 생성 (MBTI 임베딩 사용)
new_user_embedding = mbti_embeddings_dict[new_user_mbti]

weight_combinations = list(product([0.1, 0.2, 0.3, 0.4, 0.5], repeat=3))
weight_combinations = [weights for weights in weight_combinations if sum(weights) == 1.0]

best_weights = None
best_mse = float('inf')

for weights in weight_combinations:
    weight_mbti, weight_similar, weight_model = weights
    combined_recommendations = recommend_movies_combined(new_user_embedding, new_user_preferred_movies, movie_embeddings_dict,
                                                         best_gbm, weight_mbti=weight_mbti, weight_similar=weight_similar, weight_model=weight_model)
    mse = calculate_mse(combined_recommendations, movies, best_gbm, cbf_model_input_scaled)
    if mse < best_mse:
        best_mse = mse
        best_weights = weights

print(f'Best Weights: {best_weights}, Best MSE: {best_mse}')

best_weight_mbti, best_weight_similar, best_weight_model = best_weights

**최적화된 결합 추천 시스템 실행**

In [ ]:
# 3. 최적화된 결합 추천 시스템
def recommend_movies_combined_optimized(user_embedding, preferred_movies, movie_embeddings_dict, model,
                                        weight_mbti=best_weight_mbti, weight_similar=best_weight_similar, weight_model=best_weight_model, top_n=20):
    mbti_recommendations = recommend_movies_by_mbti(user_embedding, movie_embeddings_dict, top_n=100)
    similar_movie_recommendations = recommend_similar_movies(preferred_movies, movie_embeddings_dict, top_n=100)

    combined_recommendations = {}

    for movie, score in mbti_recommendations:
        if movie not in combined_recommendations:
            combined_recommendations[movie] = 0
        combined_recommendations[movie] += weight_mbti * score

    for movie, score in similar_movie_recommendations:
        if movie not in combined_recommendations:
            combined_recommendations[movie] = 0
        combined_recommendations[movie] += weight_similar * score

    movie_indices = [movies[movies['name'] == movie].index[0] for movie in combined_recommendations.keys()]
    if movie_indices:
        movie_features = cbf_model_input_scaled[movie_indices]
        model_scores = model.predict(movie_features).flatten()

        for movie, model_score in zip(combined_recommendations.keys(), model_scores):
            combined_recommendations[movie] += weight_model * model_score

    filtered_recommendations = {movie: score for movie, score in combined_recommendations.items()
                                if movies.loc[movies['name'] == movie, 'star_avg'].values[0] >= 3.3}

    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie, _ in final_recommendations[:top_n]]

In [ ]:
# 예시 사용자 입력 데이터
new_user_mbti = "INTP"
new_user_preferred_movies = ["해리포터와 불의 잔", "스파이더맨: 노 웨이 홈"]

# 예시 사용자 임베딩 생성 (MBTI 임베딩 사용)
new_user_embedding = mbti_embeddings_dict[new_user_mbti]

# 최적화된 결합 추천 시스템 실행
optimized_recommendations = recommend_movies_combined_optimized(new_user_embedding, new_user_preferred_movies, movie_embeddings_dict, best_gbm)
print("최적화된 결합 추천 영화 목록:", optimized_recommendations)

In [ ]:
labse_model = keras.models.load_model("resource/labse_model.h5")

**모델 저장 및 로드**

In [ ]:
# 모델 저장
import joblib

joblib.dump(best_gbm, '/content/best_gbm_model.pkl')
labse_model.save('/content/labse_model.h5')

In [ ]:
# import joblib
# import tensorflow as tf

# # Gradient Boosting Regressor 모델 불러오기
# best_gbm = joblib.load('/content/best_gbm_model.pkl')

# # TensorFlow LaBSE 모델 불러오기
# labse_model = tf.keras.models.load_model('/content/labse_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})

**추천 목록을 제공받고 좋아하는 콘텐츠 기반으로 재추천하는 로직**

In [ ]:
# 재추천 시스템
def recommend_movies_combined_excluding(user_embedding, preferred_movies, previous_recommendations, initial_preferred_movies, movie_embeddings_dict, model,
                                        weight_mbti=0.4, weight_similar=0.5, weight_model=0.1, top_n=20):
    mbti_recommendations = recommend_movies_by_mbti(user_embedding, movie_embeddings_dict, top_n=100)
    similar_movie_recommendations = recommend_similar_movies(preferred_movies, movie_embeddings_dict, top_n=100)

    combined_recommendations = {}

    for movie, score in mbti_recommendations:
        if movie not in previous_recommendations and movie not in preferred_movies and movie not in initial_preferred_movies:
            if movie not in combined_recommendations:
                combined_recommendations[movie] = 0
            combined_recommendations[movie] += weight_mbti * score

    for movie, score in similar_movie_recommendations:
        if movie not in previous_recommendations and movie not in preferred_movies and movie not in initial_preferred_movies:
            if movie not in combined_recommendations:
                combined_recommendations[movie] = 0
            combined_recommendations[movie] += weight_similar * score

    movie_indices = [movies[movies['name'] == movie].index[0] for movie in combined_recommendations.keys() if movie not in previous_recommendations and movie not in preferred_movies and movie not in initial_preferred_movies]
    if movie_indices:
        movie_features = cbf_model_input_scaled[movie_indices]
        model_scores = model.predict(movie_features).flatten()

        for movie, model_score in zip(combined_recommendations.keys(), model_scores):
            if movie not in previous_recommendations and movie not in preferred_movies and movie not in initial_preferred_movies:
                combined_recommendations[movie] += weight_model * model_score

    filtered_recommendations = {movie: score for movie, score in combined_recommendations.items()
                                if movie not in previous_recommendations and movie not in preferred_movies and movie not in initial_preferred_movies and movies.loc[movies['name'] == movie, 'star_avg'].values[0] >= 3.3}

    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie, _ in final_recommendations[:top_n]]

In [ ]:
# 첫 번째 추천 실행
new_user_mbti = "INFP"
new_user_preferred_movies = ["어바웃 타임", "라라랜드", "이터널 선샤인"]

# 예시 사용자 임베딩 생성 (MBTI 임베딩 사용)
new_user_embedding = mbti_embeddings_dict[new_user_mbti]
first_recommendations = recommend_movies_combined(new_user_embedding, new_user_preferred_movies, movie_embeddings_dict, best_gbm)
print("첫 번째 추천 영화 목록:", first_recommendations)

In [ ]:
# 첫 번째 추천으로부터 재추천
liked_movies_input = input("첫 번째 추천 목록에서 선호하는 영화를 쉼표로 구분하여 입력하세요: ")
liked_movies = [movie.strip() for movie in liked_movies_input.split(',')]

previous_recommendations = first_recommendations
new_preferred_movies = liked_movies  # 사용자 입력을 기반으로 새로운 선호 영화 목록 생성

second_recommendations = recommend_movies_combined_excluding(new_user_embedding, new_preferred_movies, previous_recommendations, new_user_preferred_movies, movie_embeddings_dict, best_gbm)
print("재추천 영화 목록:", second_recommendations)